In [46]:
import pandas as pd
dataset = pd.read_csv("CKD.csv")

In [47]:
dataset= pd.get_dummies(dataset,dtype=int,drop_first=True)
dataset

,age,bp,al,su,bgr,bu,sc,sod,pot,hrmo,...,pc_normal,pcc_present,ba_present,htn_yes,dm_yes,cad_yes,appet_yes,pe_yes,ane_yes,classification_yes
0,2.000000,76.459948,3.0,0.0,148.112676,57.482105,3.077356,137.528754,4.627244,12.518156,...,0,0,0,0,0,0,1,1,0,1
1,3.000000,76.459948,2.0,0.0,148.112676,22.000000,0.700000,137.528754,4.627244,10.700000,...,1,0,0,0,0,0,1,0,0,1
2,4.000000,76.459948,1.0,0.0,99.000000,23.000000,0.600000,138.000000,4.400000,12.000000,...,1,0,0,0,0,0,1,0,0,1
3,5.000000,76.459948,1.0,0.0,148.112676,16.000000,0.700000,138.000000,3.200000,8.100000,...,1,0,0,0,0,0,1,0,1,1
4,5.000000,50.000000,0.0,0.0,148.112676,25.000000,0.600000,137.528754,4.627244,11.800000,...,1,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,51.492308,70.000000,0.0,0.0,219.000000,36.000000,1.300000,139.000000,3.700000,12.500000,...,1,0,0,0,0,0,1,0,0,1
395,51.492308,70.000000,0.0,2.0,220.000000,68.000000,2.800000,137.528754,4.627244,8.700000,...,1,0,0,1,1,0,1,0,1,1
396,51.492308,70.000000,3.0,0.0,110.000000,115.000000,6.000000,134.000000,2.700000,9.100000,...,1,0,0,1,1,0,0,0,0,1
397,51.492308,90.000000,0.0,0.0,207.000000,80.000000,6.800000,142.000000,5.500000,8.500000,...,1,0,0,1,1,0,1,0,1,1


In [48]:
dataset.columns

Index(['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes', 'classification_yes'],
      dtype='object')

In [49]:
independent = dataset[['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes']]
dependent = dataset[['classification_yes']]
dependent.value_counts()

classification_yes
1                     249
0                     150
Name: count, dtype: int64

In [50]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test= train_test_split(independent,dependent,test_size=0.30,random_state=0)

In [52]:
#Support Vector Machine

from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

param_grid={'kernel':['poly','sigmoid','rbf','linear'],'C':[0.1,10,100,1000],'gamma':['scale','auto'],'verbose':[True]}

grid = GridSearchCV(SVC(probability=True),param_grid,refit=True,verbose=3,n_jobs=-1,scoring='f1_weighted')
grid.fit(X_train,Y_train)

grid_prediction = grid.predict(X_test)

from sklearn.metrics import confusion_matrix,classification_report,roc_auc_score
cm = confusion_matrix(Y_test,grid_prediction)
print("****************************************************************************")
print(cm)
print("****************************************************************************")
clf=classification_report(Y_test,grid_prediction)
print(clf)
print("****************************************************************************")
ras= roc_auc_score(Y_test,grid.predict_proba(X_test)[:,1])
print(ras)
print("****************************************************************************")

report = grid.cv_results_
table=pd.DataFrame.from_dict(report)

Fitting 5 folds for each of 32 candidates, totalling 160 fits
[LibSVM]****************************************************************************
[[45  0]
 [ 2 73]]
****************************************************************************
              precision    recall  f1-score   support

           0       0.96      1.00      0.98        45
           1       1.00      0.97      0.99        75

    accuracy                           0.98       120
   macro avg       0.98      0.99      0.98       120
weighted avg       0.98      0.98      0.98       120

****************************************************************************
0.9997037037037036
****************************************************************************


C:\Users\nravi378\AppData\Local\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [53]:
#Decision Tree

from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

param_grid={'criterion':['gini','entropy','log_loss'],'splitter':['best','random'],}

grid = GridSearchCV(DecisionTreeClassifier(),param_grid,refit=True,verbose=3,n_jobs=-1,scoring='f1_weighted')
grid = grid.fit(X_train,Y_train)

grid_prediction = grid.predict(X_test)

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(Y_test,grid_prediction)
from sklearn.metrics import classification_report
cfr=classification_report(Y_test,grid_prediction)
print("****************************************************************************")
print(cm)
print("****************************************************************************")
print(cfr)
print("****************************************************************************")


from sklearn.metrics import roc_auc_score
rac=roc_auc_score(Y_test,grid.predict_proba(X_test)[:,1])
print(rac)

dic = grid.cv_results_
table = pd.DataFrame.from_dict(dic)
table

Fitting 5 folds for each of 6 candidates, totalling 30 fits
****************************************************************************
[[45  0]
 [ 2 73]]
****************************************************************************
              precision    recall  f1-score   support

           0       0.96      1.00      0.98        45
           1       1.00      0.97      0.99        75

    accuracy                           0.98       120
   macro avg       0.98      0.99      0.98       120
weighted avg       0.98      0.98      0.98       120

****************************************************************************
0.9866666666666667


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_splitter,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.003126,0.006251,0.006249,0.007653,gini,best,"{'criterion': 'gini', 'splitter': 'best'}",0.911692,0.927778,0.894531,0.891667,0.981728,0.921479,0.032804,6
1,0.003124,0.006249,0.003126,0.006251,gini,random,"{'criterion': 'gini', 'splitter': 'random'}",0.982221,0.964572,0.894531,0.964286,0.963247,0.953771,0.030449,2
2,0.000000,0.000000,0.007558,0.009485,entropy,best,"{'criterion': 'entropy', 'splitter': 'best'}",0.964286,0.963889,0.859355,0.909115,0.963636,0.932056,0.042098,5
3,0.003125,0.006250,0.009977,0.002824,entropy,random,"{'criterion': 'entropy', 'splitter': 'random'}",0.963889,0.946153,1.000000,0.964572,0.981894,0.971302,0.018267,1
4,0.002118,0.003317,0.007664,0.003911,log_loss,best,"{'criterion': 'log_loss', 'splitter': 'best'}",0.982051,0.945469,0.876984,0.909115,0.981728,0.939069,0.041134,4
5,0.000405,0.000810,0.008123,0.004062,log_loss,random,"{'criterion': 'log_loss', 'splitter': 'random'}",0.964286,0.946153,0.894269,0.947015,0.963636,0.943072,0.025611,3


In [54]:
#Random Forest

from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

param_grid={'n_estimators':[10,100,500],'criterion':['gini','entropy','log_loss']}

grid = GridSearchCV(RandomForestClassifier(),param_grid,refit=True,verbose=3,n_jobs=-1,scoring='f1_weighted')
grid = grid.fit(X_train,Y_train)

grid_prediction = grid.predict(X_test)

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(Y_test,grid_prediction)
from sklearn.metrics import classification_report
cfr=classification_report(Y_test,grid_prediction)
print("****************************************************************************")
print(cm)
print("****************************************************************************")
print(cfr)
print("****************************************************************************")


from sklearn.metrics import roc_auc_score
rac=roc_auc_score(Y_test,grid.predict_proba(X_test)[:,1])
print(rac)

dic = grid.cv_results_
table = pd.DataFrame.from_dict(dic)
table

Fitting 5 folds for each of 9 candidates, totalling 45 fits
****************************************************************************
[[45  0]
 [ 3 72]]
****************************************************************************
              precision    recall  f1-score   support

           0       0.94      1.00      0.97        45
           1       1.00      0.96      0.98        75

    accuracy                           0.97       120
   macro avg       0.97      0.98      0.97       120
weighted avg       0.98      0.97      0.98       120

****************************************************************************
0.9986666666666666


C:\Users\nravi378\AppData\Local\anaconda3\Lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.018749,0.006252,0.006251,0.007656,gini,10,"{'criterion': 'gini', 'n_estimators': 10}",1.000000,0.946153,0.929513,0.946663,1.000000,0.964466,0.029663,9
1,0.168080,0.007158,0.014889,0.003686,gini,100,"{'criterion': 'gini', 'n_estimators': 100}",1.000000,0.982051,0.947015,0.964286,1.000000,0.978670,0.020641,3
2,0.828179,0.024059,0.060134,0.006984,gini,500,"{'criterion': 'gini', 'n_estimators': 500}",1.000000,0.982051,0.947015,0.982051,0.981728,0.978569,0.017258,4
3,0.018526,0.003484,0.006047,0.004230,entropy,10,"{'criterion': 'entropy', 'n_estimators': 10}",1.000000,0.982051,0.982221,0.964286,1.000000,0.985712,0.013364,1
4,0.176144,0.015491,0.022082,0.010117,entropy,100,"{'criterion': 'entropy', 'n_estimators': 100}",1.000000,0.945469,0.929513,0.964286,1.000000,0.967853,0.028463,7
5,0.750686,0.086610,0.040150,0.010364,entropy,500,"{'criterion': 'entropy', 'n_estimators': 500}",1.000000,0.945469,0.947015,0.982051,1.000000,0.974907,0.024310,5
6,0.023076,0.002254,0.009231,0.001549,log_loss,10,"{'criterion': 'log_loss', 'n_estimators': 10}",1.000000,0.982051,0.929513,0.982051,1.000000,0.978723,0.025881,2
7,0.163956,0.010731,0.014908,0.003679,log_loss,100,"{'criterion': 'log_loss', 'n_estimators': 100}",0.982221,0.945469,0.929513,0.982051,1.000000,0.967851,0.026125,8
8,0.714604,0.027742,0.042961,0.014223,log_loss,500,"{'criterion': 'log_loss', 'n_estimators': 500}",1.000000,0.945469,0.947015,0.982051,1.000000,0.974907,0.024310,5


In [61]:
#Logistic Regression

from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

param_grid={'solver':['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga']}

grid = GridSearchCV(LogisticRegression(),param_grid,refit=True,verbose=3,n_jobs=-1,scoring='f1_weighted')
grid = grid.fit(X_train,Y_train)

grid_prediction = grid.predict(X_test)

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(Y_test,grid_prediction)
from sklearn.metrics import classification_report
cfr=classification_report(Y_test,grid_prediction)
print("****************************************************************************")
print(cm)
print("****************************************************************************")
print(cfr)
print("****************************************************************************")


from sklearn.metrics import roc_auc_score
rac=roc_auc_score(Y_test,grid.predict_proba(X_test)[:,1])
print(rac)

dic = grid.cv_results_
table = pd.DataFrame.from_dict(dic)
table

import pickle
filename = "Final_model.sav"
pickle.dump(grid,open(filename,'wb'))

loaded_model = pickle.load(open("Final_model.sav",'rb'))
Results = loaded_model.predict([[100, 3, 5, 54,54, 67, 34, 86, 34, 56, 98,
       67, 34, 43, 65, 87, 23, 1, 0,
       1, 0,1,1,0,0,1,0]])
print(Results)


Fitting 5 folds for each of 6 candidates, totalling 30 fits
****************************************************************************
[[45  0]
 [ 1 74]]
****************************************************************************
              precision    recall  f1-score   support

           0       0.98      1.00      0.99        45
           1       1.00      0.99      0.99        75

    accuracy                           0.99       120
   macro avg       0.99      0.99      0.99       120
weighted avg       0.99      0.99      0.99       120

****************************************************************************
1.0
[0]


C:\Users\nravi378\AppData\Local\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [56]:
#KNN

from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

pagrid={'n_neighbors': [3,5,10,15],'weights':['uniform','distance'],'algorithm':['auto','ball_tree','kd_tree','brute']}

grid = GridSearchCV(KNeighborsClassifier(),pagrid,refit=True,verbose=3,n_jobs=-1,scoring='f1_weighted')
grid = grid.fit(X_train,Y_train)

grid_prediction = grid.predict(X_test)

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(Y_test,grid_prediction)
from sklearn.metrics import classification_report
cfr=classification_report(Y_test,grid_prediction)
print("****************************************************************************")
print(cm)
print("****************************************************************************")
print(cfr)
print("****************************************************************************")


from sklearn.metrics import roc_auc_score
rac=roc_auc_score(Y_test,grid.predict_proba(X_test)[:,1])
print(rac)

dic = grid.cv_results_
table = pd.DataFrame.from_dict(dic)
table

Fitting 5 folds for each of 32 candidates, totalling 160 fits


C:\Users\nravi378\AppData\Local\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:228: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


****************************************************************************
[[45  0]
 [ 5 70]]
****************************************************************************
              precision    recall  f1-score   support

           0       0.90      1.00      0.95        45
           1       1.00      0.93      0.97        75

    accuracy                           0.96       120
   macro avg       0.95      0.97      0.96       120
weighted avg       0.96      0.96      0.96       120

****************************************************************************
1.0


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_algorithm,param_n_neighbors,param_weights,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.005549,0.000719,0.126177,8.037567e-04,auto,3,uniform,"{'algorithm': 'auto', 'n_neighbors': 3, 'weigh...",0.964572,0.982221,0.929513,0.964572,1.000000,0.968176,0.023370,1
1,0.005530,0.005321,0.090091,4.196976e-02,auto,3,distance,"{'algorithm': 'auto', 'n_neighbors': 3, 'weigh...",0.964572,0.982221,0.929513,0.964572,1.000000,0.968176,0.023370,1
2,0.000000,0.000000,0.031244,0.000000e+00,auto,5,uniform,"{'algorithm': 'auto', 'n_neighbors': 5, 'weigh...",0.964572,0.964572,0.929513,0.964572,0.963912,0.957428,0.013960,13
3,0.001200,0.001600,0.033892,1.055438e-02,auto,5,distance,"{'algorithm': 'auto', 'n_neighbors': 5, 'weigh...",0.964572,0.982221,0.929513,0.964572,0.963912,0.960958,0.017180,9
4,0.001701,0.002524,0.040583,1.108395e-02,auto,10,uniform,"{'algorithm': 'auto', 'n_neighbors': 10, 'weig...",0.894531,0.894531,0.929513,0.964572,0.963912,0.929412,0.031176,21
5,0.007452,0.006677,0.034074,5.634058e-03,auto,10,distance,"{'algorithm': 'auto', 'n_neighbors': 10, 'weig...",0.964572,0.929513,0.929513,0.964572,0.963912,0.950416,0.017070,17
6,0.006651,0.007366,0.036502,7.126826e-03,auto,15,uniform,"{'algorithm': 'auto', 'n_neighbors': 15, 'weig...",0.894531,0.894531,0.912029,0.929513,0.963912,0.918903,0.025975,29
7,0.003125,0.006250,0.033064,1.479204e-03,auto,15,distance,"{'algorithm': 'auto', 'n_neighbors': 15, 'weig...",0.894531,0.912029,0.929513,0.929513,0.963912,0.925900,0.023010,25
8,0.000000,0.000000,0.015627,3.270422e-06,ball_tree,3,uniform,"{'algorithm': 'ball_tree', 'n_neighbors': 3, '...",0.964572,0.982221,0.929513,0.964572,1.000000,0.968176,0.023370,1
9,0.000000,0.000000,0.009977,8.220032e-03,ball_tree,3,distance,"{'algorithm': 'ball_tree', 'n_neighbors': 3, '...",0.964572,0.982221,0.929513,0.964572,1.000000,0.968176,0.023370,1


In [57]:
#Navies bayes


from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import BernoulliNB

param_grid = {'force_alpha':[True,False]}

grid = GridSearchCV(BernoulliNB(),param_grid,refit=True,verbose=3,n_jobs=-1,scoring='f1_weighted')
grid = grid.fit(X_train,Y_train)

grid_prediction = grid.predict(X_test)

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(Y_test,grid_prediction)
from sklearn.metrics import classification_report
cfr=classification_report(Y_test,grid_prediction)
print("****************************************************************************")
print(cm)
print("****************************************************************************")
print(cfr)
print("****************************************************************************")


from sklearn.metrics import roc_auc_score
rac=roc_auc_score(Y_test,grid.predict_proba(X_test)[:,1])
print(rac)

dic = grid.cv_results_
table = pd.DataFrame.from_dict(dic)
table

Fitting 5 folds for each of 2 candidates, totalling 10 fits
****************************************************************************
[[45  0]
 [ 3 72]]
****************************************************************************
              precision    recall  f1-score   support

           0       0.94      1.00      0.97        45
           1       1.00      0.96      0.98        75

    accuracy                           0.97       120
   macro avg       0.97      0.98      0.97       120
weighted avg       0.98      0.97      0.98       120

****************************************************************************
1.0


C:\Users\nravi378\AppData\Local\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_force_alpha,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.006101,0.002164,0.004022,0.002155,True,{'force_alpha': True},1.0,1.0,0.982221,0.982221,1.0,0.992888,0.00871,1
1,0.001800,0.001470,0.002117,0.001840,False,{'force_alpha': False},1.0,1.0,0.982221,0.982221,1.0,0.992888,0.00871,1
